# Bike Sharing Predictive Model
## predict net rate of bike renting at a selected station
### Jackie Cui

In [13]:
import pandas as pd
import numpy as np

In [92]:
trip=pd.read_csv('trip_data.csv', parse_dates=['Start Date', 'End Date'])

In [93]:
station=pd.read_csv('station_data.csv')

In [87]:
weather=pd.read_csv('weather_data.csv', parse_dates=['Date'])

In [88]:
station.head()

,Id,Name,Lat,Long,Dock Count,City
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose


In [94]:
trip.head()

,Trip ID,Start Date,Start Station,End Date,End Station,Subscriber Type
0,913460,2015-08-31 23:26:00,50,2015-08-31 23:39:00,70,Subscriber
1,913459,2015-08-31 23:11:00,31,2015-08-31 23:28:00,27,Subscriber
2,913455,2015-08-31 23:13:00,47,2015-08-31 23:18:00,64,Subscriber
3,913454,2015-08-31 23:10:00,10,2015-08-31 23:17:00,8,Subscriber
4,913453,2015-08-31 23:09:00,51,2015-08-31 23:22:00,60,Customer


In [70]:
weather.head()

,Date,Max TemperatureF,Mean TemperatureF,Min TemperatureF,Max Dew PointF,MeanDew PointF,Min DewpointF,Max Humidity,Mean Humidity,Min Humidity,...,Mean VisibilityMiles,Min VisibilityMiles,Max Wind SpeedMPH,Mean Wind SpeedMPH,Max Gust SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees,Zip
0,2014-01-09,83.0,70.0,57.0,58.0,56.0,52.0,86.0,64.0,42.0,...,10.0,8.0,16.0,7.0,20.0,0.0,0.0,NaN,290.0,94107
1,2014-02-09,72.0,66.0,60.0,58.0,57.0,55.0,84.0,73.0,61.0,...,10.0,7.0,21.0,8.0,NaN,0.0,5.0,NaN,290.0,94107
2,2014-03-09,76.0,69.0,61.0,57.0,56.0,55.0,84.0,69.0,53.0,...,10.0,10.0,21.0,8.0,24.0,0.0,4.0,NaN,276.0,94107
3,2014-04-09,74.0,68.0,61.0,57.0,57.0,56.0,84.0,71.0,57.0,...,10.0,8.0,22.0,8.0,25.0,0.0,5.0,NaN,301.0,94107
4,2014-05-09,72.0,66.0,60.0,57.0,56.0,54.0,84.0,71.0,57.0,...,9.0,7.0,18.0,8.0,32.0,0.0,4.0,NaN,309.0,94107


In [95]:
trip['Start Hour']=np.array(list(map(lambda x: x.hour, trip['Start Date'])))
trip['End Hour']=np.array(list(map(lambda x: x.hour, trip['End Date'])))
trip['Start Day']=np.array(list(map(lambda x: x.date(), trip['Start Date'])))
trip['End Day']=np.array(list(map(lambda x: x.date(), trip['End Date'])))
trip.drop(['Start Date', 'End Date'], axis=1, inplace=True)

In [96]:
trip.head()

,Trip ID,Start Station,End Station,Subscriber Type,Start Hour,End Hour,Start Day,End Day
0,913460,50,70,Subscriber,23,23,2015-08-31,2015-08-31
1,913459,31,27,Subscriber,23,23,2015-08-31,2015-08-31
2,913455,47,64,Subscriber,23,23,2015-08-31,2015-08-31
3,913454,10,8,Subscriber,23,23,2015-08-31,2015-08-31
4,913453,51,60,Customer,23,23,2015-08-31,2015-08-31


In [97]:
unique_station=np.sort(station['Id'].unique())
#np.sort(unique_station)
unique_station

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 21, 22, 23,
       24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41,
       42, 45, 46, 47, 48, 49, 50, 51, 54, 55, 56, 57, 58, 59, 60, 61, 62,
       63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 80, 82,
       83, 84, 85, 86, 87, 88, 89, 90])

In [105]:
unique_start_date = np.sort(trip['Start Day'].unique())
#unique_start_date

In [106]:
unique_end_date = np.sort(trip['End Day'].unique())
#unique_end_date

In [182]:
test = trip.groupby(['Start Station','Start Day','Start Hour']).agg('count')
test = test.reset_index()
test = test.rename(columns = {'Trip ID':'PickUp'})
start = test.iloc[:,0:4]

In [183]:
test1 = trip.groupby(['End Station','End Day','End Hour']).agg('count')
test1 = test1.reset_index()
test1 = test1.rename(columns = {'Trip ID':'DropOff', 'End Station':'Start Station','End Day':'Start Day','End Hour':'Start Hour'})
end = test1.iloc[:,0:4]

In [184]:
start.head()

,Start Station,Start Day,Start Hour,PickUp
0,2,2014-01-09,12,1
1,2,2014-01-09,21,4
2,2,2014-01-09,22,1
3,2,2014-01-10,6,1
4,2,2014-01-10,7,3


In [185]:
end.head()

,Start Station,Start Day,Start Hour,DropOff
0,2,2014-01-09,14,1
1,2,2014-01-10,7,4
2,2,2014-01-10,9,1
3,2,2014-01-10,12,1
4,2,2014-01-10,14,1


In [186]:
test_join=start.merge(end, how='outer', on=['Start Station', 'Start Day', 'Start Hour'])
test_join=test_join.fillna(0)
test_join['Usage']=abs(test_join['PickUp']-test_join['DropOff'])
test_join.head()

,Start Station,Start Day,Start Hour,PickUp,DropOff,Usage
0,2,2014-01-09,12,1.0,0.0,1.0
1,2,2014-01-09,21,4.0,0.0,4.0
2,2,2014-01-09,22,1.0,0.0,1.0
3,2,2014-01-10,6,1.0,0.0,1.0
4,2,2014-01-10,7,3.0,4.0,1.0


In [198]:
test_city = pd.merge(test_join, station, left_on='Start Station', right_on='Id')

In [199]:
test_city = test_city.drop(['Id', 'Lat','Long','Dock Count'], axis=1)
test_city = test_city.rename(columns = {'Start Station':'Station','Start Day':'Date','Start Hour':'Hour'})
test_city.head()

,Station,Date,Hour,PickUp,DropOff,Usage,Name,City
0,2,2014-01-09,12,1.0,0.0,1.0,San Jose Diridon Caltrain Station,San Jose
1,2,2014-01-09,21,4.0,0.0,4.0,San Jose Diridon Caltrain Station,San Jose
2,2,2014-01-09,22,1.0,0.0,1.0,San Jose Diridon Caltrain Station,San Jose
3,2,2014-01-10,6,1.0,0.0,1.0,San Jose Diridon Caltrain Station,San Jose
4,2,2014-01-10,7,3.0,4.0,1.0,San Jose Diridon Caltrain Station,San Jose


In [200]:
select=['Date', 'Mean TemperatureF','MeanDew PointF', 'Mean Humidity', 'Mean Sea Level PressureIn' , 'Mean VisibilityMiles', 'Mean Wind SpeedMPH'
       , 'PrecipitationIn',  'CloudCover', 'WindDirDegrees', 'Zip']
weather_test=weather[select]
weather_test.head()


,Date,Mean TemperatureF,MeanDew PointF,Mean Humidity,Mean Sea Level PressureIn,Mean VisibilityMiles,Mean Wind SpeedMPH,PrecipitationIn,CloudCover,WindDirDegrees,Zip
0,2014-01-09,70.0,56.0,64.0,29.82,10.0,7.0,0.0,0.0,290.0,94107
1,2014-02-09,66.0,57.0,73.0,29.82,10.0,8.0,0.0,5.0,290.0,94107
2,2014-03-09,69.0,56.0,69.0,29.76,10.0,8.0,0.0,4.0,276.0,94107
3,2014-04-09,68.0,57.0,71.0,29.76,10.0,8.0,0.0,5.0,301.0,94107
4,2014-05-09,66.0,56.0,71.0,29.87,9.0,8.0,0.0,4.0,309.0,94107


In [228]:
weather_test['Zip'].unique()
weather_city=weather_test.replace([94107, 94063, 94301, 94041, 95113], ['San Francisco','Redwood City', 'Palo Alto' ,'Mountain View', 'San Jose'])
weather_city = weather_city.rename(columns = {'Zip':'City'})
weather_city.head()

,Date,Mean TemperatureF,MeanDew PointF,Mean Humidity,Mean Sea Level PressureIn,Mean VisibilityMiles,Mean Wind SpeedMPH,PrecipitationIn,CloudCover,WindDirDegrees,City
0,2014-01-09,70.0,56.0,64.0,29.82,10.0,7.0,0.0,0.0,290.0,San Francisco
1,2014-02-09,66.0,57.0,73.0,29.82,10.0,8.0,0.0,5.0,290.0,San Francisco
2,2014-03-09,69.0,56.0,69.0,29.76,10.0,8.0,0.0,4.0,276.0,San Francisco
3,2014-04-09,68.0,57.0,71.0,29.76,10.0,8.0,0.0,5.0,301.0,San Francisco
4,2014-05-09,66.0,56.0,71.0,29.87,9.0,8.0,0.0,4.0,309.0,San Francisco


In [229]:
test_city['Date'] = pd.to_datetime(test_city['Date'])
test_city.info()
test_city.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198156 entries, 0 to 198155
Data columns (total 8 columns):
Station    198156 non-null int64
Date       198156 non-null datetime64[ns]
Hour       198156 non-null int64
PickUp     198156 non-null float64
DropOff    198156 non-null float64
Usage      198156 non-null float64
Name       198156 non-null object
City       198156 non-null object
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 13.6+ MB


,Station,Date,Hour,PickUp,DropOff,Usage,Name,City
0,2,2014-01-09,12,1.0,0.0,1.0,San Jose Diridon Caltrain Station,San Jose
1,2,2014-01-09,21,4.0,0.0,4.0,San Jose Diridon Caltrain Station,San Jose
2,2,2014-01-09,22,1.0,0.0,1.0,San Jose Diridon Caltrain Station,San Jose
3,2,2014-01-10,6,1.0,0.0,1.0,San Jose Diridon Caltrain Station,San Jose
4,2,2014-01-10,7,3.0,4.0,1.0,San Jose Diridon Caltrain Station,San Jose


In [230]:
final=test_city.merge(weather_city, how='inner', on=['Date','City'])

In [231]:
final.head()

,Station,Date,Hour,PickUp,DropOff,Usage,Name,City,Mean TemperatureF,MeanDew PointF,Mean Humidity,Mean Sea Level PressureIn,Mean VisibilityMiles,Mean Wind SpeedMPH,PrecipitationIn,CloudCover,WindDirDegrees
0,2,2014-01-09,12,1.0,0.0,1.0,San Jose Diridon Caltrain Station,San Jose,72.0,54.0,59.0,29.81,10.0,5.0,0.0,0.0,296.0
1,2,2014-01-09,21,4.0,0.0,4.0,San Jose Diridon Caltrain Station,San Jose,72.0,54.0,59.0,29.81,10.0,5.0,0.0,0.0,296.0
2,2,2014-01-09,22,1.0,0.0,1.0,San Jose Diridon Caltrain Station,San Jose,72.0,54.0,59.0,29.81,10.0,5.0,0.0,0.0,296.0
3,2,2014-01-09,14,0.0,1.0,1.0,San Jose Diridon Caltrain Station,San Jose,72.0,54.0,59.0,29.81,10.0,5.0,0.0,0.0,296.0
4,3,2014-01-09,12,1.0,0.0,1.0,San Jose Civic Center,San Jose,72.0,54.0,59.0,29.81,10.0,5.0,0.0,0.0,296.0


In [219]:
weather_city.Date

0       2014-01-09
1       2014-02-09
2       2014-03-09
3       2014-04-09
4       2014-05-09
           ...    
1820    2015-08-27
1821    2015-08-28
1822    2015-08-29
1823    2015-08-30
1824    2015-08-31
Name: Date, Length: 1825, dtype: object

In [220]:
test_city.Date

0         2014-01-09
1         2014-01-09
2         2014-01-09
3         2014-01-10
4         2014-01-10
             ...    
198151    2015-12-07
198152    2015-12-07
198153    2015-12-07
198154    2015-12-08
198155    2015-12-08
Name: Date, Length: 198156, dtype: object